In [4]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

# # Print statements to confirm that all imports were successful
# print(f"Pandas version: {pd.__version__} - Imported successfully!")
# print(f"NumPy version: {np.__version__} - Imported successfully!")
# print(f"Keras version: {keras.__version__} - Imported successfully!")
# print(f"TensorFlow version: {tf.__version__} - Imported successfully!")

# # Additional check for imported components
# tokenizer = Tokenizer()
# print("Tokenizer from tensorflow.keras - Imported successfully!")

# sequence_test = pad_sequences([[1, 2, 3], [4, 5, 6]], padding='post')
# print("pad_sequences from keras.preprocessing.sequence - Imported successfully!")

# label_encoder = LabelEncoder()
# print("LabelEncoder from sklearn.preprocessing - Imported successfully!")

# train_test = train_test_split([1, 2, 3], [4, 5, 6], test_size=0.2)
# print("train_test_split from sklearn.model_selection - Imported successfully!")

# model = Sequential()
# model.add(Embedding(input_dim=1000, output_dim=64, input_length=10))
# model.add(Flatten())
# model.add(Dense(1))
# print("Keras Sequential Model, Embedding, Flatten, and Dense layers - Imported successfully!")



data = pd.read_csv("/workspaces/Text_emotion_detection/data/train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [5]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [6]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [7]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [8]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [9]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences,  one_hot_labels, test_size=0.2)

In [10]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10


/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


400/400 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.3824 - loss: 1.5169 - val_accuracy: 0.6950 - val_loss: 0.8452
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.8619 - loss: 0.4394 - val_accuracy: 0.8188 - val_loss: 0.5493
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.9801 - loss: 0.0786 - val_accuracy: 0.8238 - val_loss: 0.5618
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9936 - loss: 0.0270 - val_accuracy: 0.8247 - val_loss: 0.5863
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9965 - loss: 0.0172 - val_accuracy: 0.8209 - val_loss: 0.6456
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.9970 - loss: 0.0152 - val_accuracy: 0.8188 - val_loss: 0.6865
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.9965 - loss: 0.0156 - val_accuracy: 0.8206 - val_loss: 0.7177
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.9978 - loss: 0.0113 - val_accuracy: 0.8

In [19]:
input_text = "i feel so scared like everyday of my life"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['fear']
